In [1]:
import pandas as pd
import torch
import os
import sys
from torch.utils.data import DataLoader
import time
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dataset_PAMAP2 import dataset_PAMAP2

In [2]:
no_id_list=[108,109]
dataset=dataset_PAMAP2(no_id_list)

In [3]:
#import importlib
#importlib.reload(dataset_PAMAP2)

In [5]:
activity_list=[1,2,3,4,5,6,7,9,10,11,12,13,16,17,18,19,20,24] #[3,4,12,13]
id_test=103
batch_size=64
stepsize=32
train_data,test_data=dataset.load_as_DataLoader_with_all_attributes(activity_list,id_test,stepsize,batch_size)

In [49]:
import matplotlib.pyplot as plt
import numpy as np
def plot_accuracies(test_acc_List,stepsize):
    res=test_acc_List
    fig=plt.figure(figsize=(10, 6))
    ax=plt.subplot(polar=True)
    
    ax.set_ylim(0.4,1)
    ax.set_yticks(np.arange(0.5,1,0.1))
    
    
    id_tests=[101,102,103,104,105,106,107]

    res.append(res[0])
    angles=[i*360/len(id_tests) for i in range(len(id_tests))]
    lines, labels = plt.thetagrids(angles, (id_tests))

    theta = np.linspace(0, 2 * np.pi, len(res))
    plt.plot(theta, res)

    #plt.fill(theta, actual, 'b', alpha=0.1)
    #plt.legend(labels=('Actual'), loc=1)
    #plt.legend(labels =('Actual', 'Expected'),loc = 1)
    
    plt.title("Step = "+str(stepsize))
    plt.show(fig)
    fig.savefig("18_activities_Step_"+str(stepsize)+".png")
    plt.close(fig) 

# Apprentissage

In [6]:
import models
import importlib
importlib.reload(models)
import pytorch_lightning as pl 
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [7]:
def train_model(model, clipping, num_epochs, data_loader_train, data_loader_val, data_loader_test):

    early_stop_callback = EarlyStopping(
      monitor='valid_accuracy',
      min_delta=0.00,
      patience=20,
      verbose=False,
      mode='max'
    )
    trainer = Trainer(max_epochs=num_epochs, gradient_clip_val= clipping, progress_bar_refresh_rate = 40, gpus=1, callbacks=[early_stop_callback])
    trainer.fit(model, data_loader_train, data_loader_val)
    final_results = trainer.logged_metrics
    final_results["test"] = trainer.test(model, data_loader_test)[0]
    final_results["num_epochs"] = trainer.current_epoch
    final_results["num_parameters"] = model.num_paramaters
    return final_results

In [75]:
#model = models.MultiChannelBase(40, [5,5], 256, len(activity_list), lr = 0.001, betas = (0.9, 0.999), eps = 1e-8)

In [76]:
#res=train_model(model, 1, 50, train_data,test_data,test_data)

In [79]:
window_sizes=[256,128,64]
kernel_sizes=[[3,3,3],[3,3],[3]]
model = models.MultiChannelMultiTime(40,window_sizes, kernel_sizes, len(activity_list), lr = 0.001, betas = (0.9, 0.999), eps = 1e-8)

In [80]:

res=train_model(model, 1, 50, train_data,test_data,test_data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | train_acc  | Accuracy   | 0     
1 | valid_acc  | Accuracy   | 0     
2 | test_acc   | Accuracy   | 0     
3 | classifier | Sequential | 72.7 K
------------------------------------------
72.7 K    Trainable params
0         Non-trainable params
72.7 K    Total params
0.291     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6280992031097412,
 'test_accuracy_epoch': 0.6280992031097412,
 'test_batch_time': 0.0059833526611328125,
 'test_batch_time_epoch': 0.006799916736781597,
 'test_loss': 3.70149564743042,
 'test_loss_epoch': 3.510043144226074}
--------------------------------------------------------------------------------


In [16]:
importlib.reload(models)
window_sizes=[256,128,64]
kernel_sizes=[3]
down_sampling_kernel=[[3,3],[3],[]]
model = models.MultiChannelMultiTimeDownSample(40,window_sizes,down_sampling_kernel, kernel_sizes,64, len(activity_list), lr = 0.0001, betas = (0.9, 0.999), eps = 1e-8)


In [ ]:
model.classifier
print()
res=train_model(model, 1, 50, train_data,test_data,test_data)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | train_acc  | Accuracy   | 0     
1 | valid_acc  | Accuracy   | 0     
2 | test_acc   | Accuracy   | 0     
3 | classifier | Sequential | 71.3 K
------------------------------------------
71.3 K    Trainable params
0         Non-trainable params
71.3 K    Total params
0.285     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [61]:
activity_list=[1,2,3,4,5,6,7,9,10,11,12,13,16,17,18,19,20,24] #[3,4,12,13]
batch_size=32
batch_sizes={8:256,16:128,32:64,64:32,128:16}


id_tests=[101,102,103,104,105,106,107]
stepsizes=[128,64,32,16,8]
results=[]
for stepsize in stepsizes:
    for id_test in id_tests:
        
        train_data,test_data=dataset.load_as_DataLoader_with_all_attributes(activity_list,id_test,stepsize,batch_sizes[stepsize])
        model = models.MultiChannelBase(40, [5,5], 256, len(activity_list), lr = 0.001, betas = (0.9, 0.999), eps = 1e-8)
        res=train_model(model, 1, 50, train_data,test_data,test_data)
        print("stepsize : ",stepsize," id_test : ",id_test," Test Accuracy : ",res['test']['test_accuracy'])
        results.append(res['test']['test_accuracy'])
        del model
        torch.cuda.empty_cache()
    plot_accuracies(results,stepsize)
    print(results)
    results=[]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | train_acc  | Accuracy   | 0     
1 | valid_acc  | Accuracy   | 0     
2 | test_acc   | Accuracy   | 0     
3 | conv       | Sequential | 16.2 K
4 | classifier | Sequential | 46.1 K
------------------------------------------
62.3 K    Trainable params
0         Non-trainable params
62.3 K    Total params
0.249     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7345041036605835,
 'test_accuracy_epoch': 0.7345041036605835,
 'test_batch_time': 0.004985809326171875,
 'test_batch_time_epoch': 0.0043748351745307446,
 'test_loss': 5.272700309753418,
 'test_loss_epoch': 7.337461471557617}
--------------------------------------------------------------------------------
stepsize :  128  id_test :  101  Test Accuracy :  0.7345041036605835


KeyboardInterrupt: 